In [1]:
!pip3 install implicit

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
data = pd.read_csv('/content/drive/MyDrive/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [6]:
result_2 = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_2.columns=['user_id', 'actual']
result_2.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [7]:
# Составить датафрейм из количества покупок товара
popularity = data_train.groupby('item_id')['user_id'].nunique().reset_index()
popularity.head(3)

,item_id,user_id
0,25671,3
1,26081,1
2,26093,1


In [8]:
# Составить датафрейм из количества покупок товара и прологорифмируем его
popularity['user_id'] = popularity['user_id'].apply(lambda x: np.round(np.log(x), 13))
popularity.head(3)

,item_id,user_id
0,25671,1.098612
1,26081,0.000000
2,26093,0.000000


In [9]:
# Сделаем L1-норализацию
log_sales_sum = np.sum(popularity['user_id'])
popularity['user_id'] = popularity['user_id'].apply(lambda x: np.divide(x, log_sales_sum))
popularity.head(3)

,item_id,user_id
0,25671,0.00001
1,26081,0.00000
2,26093,0.00000


In [10]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    recs = np.random.choice(items_weights['item_id'], size=n, replace=False, p=items_weights['user_id'] )
    
    
    return recs.tolist()

In [11]:
%%time

result_2['weighted_random_recommendation'] = result_2['user_id'].apply(lambda x: weighted_random_recommendation(popularity, n=5))

CPU times: user 2.82 s, sys: 51.1 ms, total: 2.87 s
Wall time: 2.87 s


In [12]:
result_2.head(2)

,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[10356251, 13190068, 10456237, 9526624, 930209]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1093730, 1042697, 936851, 6919453, 1112417]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [13]:
result = pd.read_csv('/content/drive/MyDrive/predictions_basic.csv', sep=',')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[9485371, 17104661, 9837998, 12171628, 13777104]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[13910038, 48728, 13416521, 12263605, 828300]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [14]:
def change_line(line):
  line = line.replace('[', '').replace(']', '').replace(' ', '')
  line = line.split(',')
  line = [int(i) for i in line]
  return line

In [15]:
coulumns = result.drop(columns=['user_id', 'actual']).columns.values.tolist()
for i in coulumns:
    result[i] = result.apply(lambda x: change_line(x[i]), axis=1)

In [16]:
result['actual'] = result_2['actual']

In [17]:
result['weighted_random_recommendation'] = result_2['weighted_random_recommendation']
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9485371, 17104661, 9837998, 12171628, 13777104]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[10356251, 13190068, 10456237, 9526624, 930209]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13910038, 48728, 13416521, 12263605, 828300]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[1093730, 1042697, 936851, 6919453, 1112417]"


In [18]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [19]:
coulumns = result.drop(columns=['user_id', 'actual']).columns.values.tolist()
for i in coulumns:
    precision = result.apply(lambda row: precision_at_k(row[i], row['actual']), axis=1).mean()
    print(f'{i}: {precision*100}%')


random_recommendation: 0.029382957884427036%
popular_recommendation: 15.523996082272081%
itemitem: 13.692458374142857%
cosine: 13.290891283055686%
tfidf: 13.898139079333829%
own_purchases: 17.969311132876015%
weighted_random_recommendation: 0.1077375122428991%


Наиболее эффективным оказался метод ItemItemRecommender с одним ближаешим соседом. Взвешаный случайным метод показал себя немного лучше обычной случайной выборке товаров.

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [27]:
# your_code